In [ ]:
#Creating the Memory class with methods for storing, loading, printing, and selecting from memory.
class Memory:

    def __init__(self):
        #This is the memory bank that holds all addresses and values in main memory
        self.memory_bank = {}
        for i in range(0, 256, 4):
            self.memory_bank[i] = None
        self.cache_is_active = False

    def store_into_memory(self, address, data):
        self.memory_bank[address] = data
        #This stores values into an address in main memory

    def load_from_memory(self, address):
        return self.memory_bank[address]
        #This returns a specified value from a memory address.

    def print_memory(self):
        for address, data in self.memory_bank.items():
            print (f"{address}:{data}")

    def select_from_available_memory(self):
        for i in self.memory_bank:
            if self.memory_bank[i] == None:
                return i
            else:
                continue
        return ("No available slots left in memory.")
    
    def load_instr_to_memory(self, file):
        list_of_instructions = []
        with open(file) as instruction_file:
            list_of_instructions += instruction_file.readlines()
        for instruction in list_of_instructions:
            selected_instruction = Instruction(instruction)
            self.store_into_memory(self.select_from_available_memory, selected_instruction)